In [ ]:
#Slides

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Normalize the data

x_train, x_test = x_train / 255.0, x_test / 255.0
# Eaxh pixel is now between [0,1]

# Conv2D model via class definiton
Based on : https://www.tensorflow.org/tutorials/quickstart/advanced
However, there are a few changes like adding a pooling layer

In [ ]:
# Convulation model using the Class Definition

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation = 'softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        #print(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
# Reshape data and Put data in batches
import numpy as np
convx_train =  np.reshape(x_train, (x_train.shape[0], 28,28))
convx_test =  np.reshape(x_test, (x_test.shape[0], 28,28))

convx_train = convx_train[..., tf.newaxis].astype("float32")
convx_test = convx_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices(
    (convx_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((convx_test, y_test)).batch(32)

In [ ]:
 EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.0440097451210022, Accuracy: 98.65499877929688, Test Loss: 0.07283255457878113, Test Accuracy: 97.58999633789062
Epoch 2, Loss: 0.028148064389824867, Accuracy: 99.09333801269531, Test Loss: 0.04062037542462349, Test Accuracy: 98.61000061035156
Epoch 3, Loss: 0.01839366927742958, Accuracy: 99.41333770751953, Test Loss: 0.048590850085020065, Test Accuracy: 98.50999450683594
Epoch 4, Loss: 0.012656074948608875, Accuracy: 99.57833099365234, Test Loss: 0.04756646230816841, Test Accuracy: 98.56999969482422
Epoch 5, Loss: 0.008707458153367043, Accuracy: 99.70166778564453, Test Loss: 0.04758112505078316, Test Accuracy: 98.58000183105469


# TODO: Adding Conv to day 3 model

In [ ]:
#Initilaze the model
cnn_model = tf.keras.models.Sequential()

In [ ]:
#@title
cnn_model.add(Conv2D(32, 3, activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Here's the day 3 model
# Add a convolution layer and a pooling layer to this model

# Convert the 28x28 image to 1x764 vector
cnn_model.add(tf.keras.layers.Flatten())

# Layer of 128 neurons with sigmoid acitvation function
cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.2))

# Output layer

cnn_model.add(tf.keras.layers.Dense(10, activation='softmax'))



In [ ]:
# Compile the model so we can use that
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
cnn_model.fit(convx_train, y_train, epochs=5, validation_data=(convx_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.1777 - accuracy: 0.9474 - val_loss: 0.0613 - val_accuracy: 0.9803
Epoch 2/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0670 - accuracy: 0.9793 - val_loss: 0.0439 - val_accuracy: 0.9837
Epoch 3/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0467 - accuracy: 0.9855 - val_loss: 0.0406 - val_accuracy: 0.9861
Epoch 4/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0344 - accuracy: 0.9891 - val_loss: 0.0435 - val_accuracy: 0.9854
Epoch 5/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0265 - accuracy: 0.9916 - val_loss: 0.0391 - val_accuracy: 0.9877


# RNN/LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


rnn_model = Sequential()
rnn_model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
rnn_model.add(Dropout(0.2))

rnn_model.add(LSTM(128, activation='relu'))
rnn_model.add(Dropout(0.1))

rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dropout(0.2))

rnn_model.add(Dense(10, activation='softmax'))


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

rnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn_model.fit(x_train,
          y_train,
          epochs=3,
          validation_data=(x_test, y_test))


Epoch 1/3
1875/1875 [==============================] - 136s 71ms/step - loss: 0.5997 - accuracy: 0.8054 - val_loss: 0.1314 - val_accuracy: 0.9631
Epoch 2/3
1875/1875 [==============================] - 134s 72ms/step - loss: 0.1620 - accuracy: 0.9567 - val_loss: 0.0953 - val_accuracy: 0.9713
Epoch 3/3
1875/1875 [==============================] - 133s 71ms/step - loss: 0.1146 - accuracy: 0.9697 - val_loss: 0.0729 - val_accuracy: 0.9783


#Conclusion
As a closing point,
Try playing arounf with differetn architectures. 

Try some other dataset and see which architecture works the best

Other resources: 
Tensorflow Tutorials:
https://www.tensorflow.org/tutorials

Sentdex Tutorials: 

https://pythonprogramming.net/machine-learning-tutorials/

